In [1]:
# import warnings
# warnings.filterwarnings('ignore')
import torch
torch.cuda.empty_cache()

In [4]:
# Importing libraries
import re
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from nltk.translate.bleu_score import sentence_bleu
import os
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer



In [6]:
# Class for initializing training data
class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        if path.endswith('.xlsx'):
            self.data = pd.read_excel(path)
        elif path.endswith('.xlsx'):
            self.data = pd.read_excel(path)
        self.X = []

        for idx, row in self.data.iterrows():
            question = str(row['question'])
            answer = str(row['answer'])
            self.X.append('<startofstring> ' + question + ' <bot>: ' + answer + ' <endofstring')

        self.X_encoded = tokenizer(self.X, max_length=70, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

# Calculate BLEU score
def cal_epoch_bleu(model, tokenizer, device):
    df_eval = pd.read_excel('eval_full_chatbot.xlsx')
    bleu_scores = []
    
    for index, row in df_eval.iterrows():        
        output = infer(row['question'], model, tokenizer, device)

        bleu_score = sentence_bleu([row['answer']], output)
        bleu_scores.append(bleu_score)
    
    return bleu_scores


# Function to train model with early stopping based on BLEU score
def train(chatData, model, tokenizer, optimizer, device, model_path, epochs=100, patience=10):
    loss_values = []  # To store loss values
    bleu_scores = []  # To store BLEU scores
    best_loss = float('inf')  # Initialize best loss with positive infinity
    no_improvement_count = 0

    for i in tqdm(range(epochs)):
        total_loss = 0

        for data in chatData:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            optimizer.zero_grad()
            output = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = output.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Calculate BLEU scores for the epoch
        epoch_bleu_scores = cal_epoch_bleu(model, tokenizer, device)
        average_epoch_bleu_score = sum(epoch_bleu_scores) / len(epoch_bleu_scores)
        bleu_scores.append(average_epoch_bleu_score)

        # Log and print the loss for the epoch
        print(f"Epoch {i + 1}/{epochs} - Loss: {total_loss:.4f} - BLEU Score: {average_epoch_bleu_score:.4f}")

        # Check for early stopping based on loss
        if total_loss < best_loss:
            best_loss = total_loss
            no_improvement_count = 0
            # Save the model and its configuration after each epoch
            model.save_pretrained(model_path)  # Save both weights and configuration
            tokenizer.save_pretrained(model_path)  # Save tokenizer's configuration
        else:
            no_improvement_count += 1

        if no_improvement_count >= patience:
            print(f"No improvement in loss for {patience} consecutive epochs. Early stopping.")
            break

        # Append the loss values to the list for plotting
        loss_values.append(total_loss)

    # Plot loss and save the image
    epochs_range = range(1, len(loss_values) + 1)

    # Ensure lengths match before plotting
    min_len = min(len(epochs_range), len(bleu_scores))

    plt.figure(figsize=(18, 10))
    plt.plot(epochs_range[:min_len], loss_values[:min_len], label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training Loss')
    plt.tight_layout()
    plt.savefig('loss_final.png')  # Save the loss plot as an image
    plt.close()

    # Plot BLEU scores and save the image
    plt.figure(figsize=(18, 10))
    plt.plot(epochs_range[:min_len], bleu_scores[:min_len], label='BLEU Score', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('BLEU Score')
    plt.legend()
    plt.title('BLEU Scores')
    plt.tight_layout()
    plt.savefig('bleu_score_final.png')  # Save the BLEU score plot as an image
    plt.close()

    print("Training completed.")


# Function for model inference
def infer(inp, model, tokenizer, device):
    inp = '<startofstring> ' + inp + ' <bot>: '
    inp = tokenizer(inp, return_tensors='pt')
    X = inp['input_ids'].to(device)
    a = inp['attention_mask'].to(device)

    output = model.generate(X, attention_mask=a, max_length=50)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    output = re.split('<bot>:', output)[-1].strip()
    output = re.split('<end', output, 1)[0].strip()

    return output


In [7]:
import warnings
warnings.filterwarnings('ignore')
torch.cuda.empty_cache()

def main():
    # Path to save the trained model
    MODEL_PATH = 'chatbot_final_model.pt'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.add_special_tokens({'pad_token': '<pad>',
                                'bos_token': '<startofstring>',
                                'eos_token': '<endofstring>'})
    tokenizer.add_tokens(['<bot>:'])

    # Load or initialize the model
    if os.path.exists(MODEL_PATH):
        model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=MODEL_PATH, local_files_only=True)
    else:
        model = GPT2LMHeadModel.from_pretrained('gpt2')
        model.resize_token_embeddings(len(tokenizer))

    model = model.to(device)

    chatData = ChatData('Final_dataset.xlsx', tokenizer)
    chatData = DataLoader(chatData, batch_size=16)
    model.train()

    optimizer = Adam(model.parameters(), lr=1e-3)

    print("Training ...")
    train(chatData, model, tokenizer, optimizer, device, MODEL_PATH)

if __name__ == "__main__":
    main()


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50261. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Training ...


  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Epoch 1/100 - Loss: 657.9706 - BLEU Score: 0.0778


...

Epoch 45/100 - Loss: 13.5803 - BLEU Score: 0.6604


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Epoch 46/100 - Loss: 13.6848 - BLEU Score: 0.6581


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Epoch 47/100 - Loss: 13.8560 - BLEU Score: 0.6612
No improvement in loss for 10 consecutive epochs. Early stopping.
Training completed.


In [11]:
# Importing libraries
import os
import warnings

import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer

# from helper import infer

warnings.filterwarnings('ignore')
torch.cuda.empty_cache()

def infer(inp, model, tokenizer, device):
    inp = '<startofstring> ' + inp + ' <bot>: '
    inp = tokenizer(inp, return_tensors='pt')
    X = inp['input_ids'].to(device)
    a = inp['attention_mask'].to(device)

    output = model.generate(X, attention_mask=a, max_length=60)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    
    output = re.split('<bot>:', output)[-1].strip()
    output = re.split('<end', output, 1)[0].strip()

    return output

def main():
    # Path to save the trained model
    MODEL_PATH = 'chatbot_final_model.pt'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.add_special_tokens({'pad_token': '<pad>',
                                'bos_token': '<startofstring>',
                                'eos_token': '<endofstring>'})
    tokenizer.add_tokens(['<bot>:'])

    # Load the model
    if os.path.exists(MODEL_PATH):
        model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=MODEL_PATH, local_files_only=True)
        model = model.to(device)
        model.eval()

        print('Chat with the chatbot. Type "exit" to end the conversation.')
        while True:
            inp = input('You: ')
            if inp == 'exit':
                break
            response = infer(inp, model, tokenizer, device)
            print(f'Chatbot: {response}')
    else:
        print('Error: Model not found at location', MODEL_PATH)

if __name__ == "__main__":
    main()

Chat with the chatbot. Type "exit" to end the conversation.
You: what happens if my pr card expires?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: Expired permanent resident (PR) cards remain valid in Canada, but require renewal or replacement for provincial services, commercial vehicle return, and travel outside the country.
You: how do i renew my permanent resident card?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: If your PR card is expired or nearing expiration, you can apply for a new one, valid for 5 years, requiring permanent residency and Canadian application submission.
You: can i lay off or fire a foreign worker?


...

Chatbot: To extend or change your work permit as an International Experience Canada participant, you must apply at least 30 days before your work permit expires.
You: what about in quebec?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: A message in my account says that your profile has been updated. There are three main rules: Go to the main page of your account, Scroll down to “Continue my profile / application, not yet submitted”, Choose the line with your Express
You: can i apply for eta for my child?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: To apply for your child, submit a separate application form with specific information. Answer "yes" to "Are you applying on behalf of someone?", complete Parent/guardian details, and enter your child's information in the Contact Details section.
You: exit
